# 导包

In [1]:
import io
import os
import pandas as pd
import numpy as np
from pdfminer.converter import TextConverter
from pdfminer.pdfdocument import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from tqdm import tqdm
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
from FlagEmbedding import FlagModel
from sklearn.metrics.pairwise import cosine_similarity

d:\anaconda\envs\bdci2024\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# PDF解析函数
def parsePDF(PDF_path):
    flag = False
    if 'AZ' in PDF_path:
        flag = True
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager,fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager,converter)
    with open(PDF_path,'rb') as fh:
        for n_page,page in enumerate(PDFPage.get_pages(fh,caching=True,check_extractable=False)):
            if flag:
                if n_page < 2:
                    continue
            page_interpreter.process_page(page)
        text = fake_file_handle.getvalue()
    converter.close()
    fake_file_handle.close()
    if text:
        return text
    
# 从文本中删除指定字符串
def drop_content_from_text(text,drop_content:list):
    for content in drop_content:
        text = text.replace(content,'')
    return text

In [3]:
document_root = r'data/A榜/A_document'
output_path = r'./result.csv'
test_path = r'data/A榜/A_question.csv'

In [4]:
# 所有文档
docs = os.listdir(document_root)
parsePDF(os.path.join(document_root,docs[0]))

'本文档为2024CCFBDCI比赛用语料的一部分。部分文档使用大语言模型改写生成，内容可能与现实情况不符，可能不具备现实意义，仅允许在本次比赛中使用。【新征程上的铺路人、赋能者、护航员】系列报道之二十二：砥砺铸秋实风劲更远航——记中国联通2023年度集团级劳模风采发布时间：2024-01-10发布人：新闻宣传中心2023年，中国联通涌现出一大批先进模范人物，他们奋斗在不同岗位，有的大胆创新攻坚克难，寻求尖端技术突破，用领先科技赋能数字化建设；有的不断钻研专业技能，提高自身业务水平，用实际行动守护万家通信……他们来自天南海北，却都闪耀着一样的联通红，积极投身于以数字化网络化智能化助力中国式现代化的伟大实践，在平凡岗位上创造不凡业绩。挑最重的担子啃最硬的骨头联通华盛电商分公司办公室内的灯火彻夜不熄，这已经成为常态。在这个夏夜的凌晨12点钟，APP渠道负责人张晨正忙碌地在办公室中穿梭，她的步伐坚定有力，神情专注而严肃。为了618大促活动的顺利上线，她和团队成员们已经连续多日加班加点，几乎没有休息。张晨一边仔细审核着每一项产品政策，确保其准确无误，一边与供应商保持紧密联系，确认货源情况，以防止任何供应链上的问题可能影响到活动的正常进行。这种片刻不停歇的繁忙状态并没有让她感到疲惫，反而让她感到安心。“忙\x0c起来，才安心，”这是张晨常挂在嘴边的一句话。这不仅反映了她对工作的投入和责任心，也展示了她对团队成员们的激励和鼓舞。2021年7月，华盛电商应集团要求承接中国联通APP终端集约化运营项目。这对于张晨和她的团队来说，是一个巨大的挑战也是一次难得的机遇。联通APP作为日活跃用户超千万的官方线上平台，如何提升用户体验，让其更加便捷、智能，这是运营者们必须攻克的难题。面对这一重任，张晨并没有退缩。相反，她迎难而上，展现出了非凡的领导力和决心。她明白，这不仅是一次职业上的考验，更是一次证明自己能力的机会。“这既是机遇也是挑战，不管多难都要把这块硬骨头给啃下来！”她在团队会议上坚定地说道。为了提升终端运营效果，张晨带领团队进行了全面的流程优化。他们加班加点，反复进行体验测试，逐一对标市场上其他优秀的终端应用，找出差距并加以改进。优化终端购买过程中可能出现的各种问题，是一项复杂且繁琐的工作，但张晨和她的团队始终保持高度的专业性和热情。在她的带领下，团队成员们齐心协力，攻坚克难，实现

In [5]:
# 将所有文档内容拼接在一起
knowledge = []
drop_content = ['\x0c',
                '本文档为2024CCFBDCI比赛用语料的一部分。部分文档使用大语言模型改写生成，内容可能与现实情况不符，可能不具备现实意义，仅允许在本次比赛中使用。']
for doc in tqdm(docs):
    doc_path = os.path.join(document_root,doc)
    text = drop_content_from_text(parsePDF(doc_path),drop_content)
    knowledge.append(text)
knowledge = ''.join(knowledge)

100%|██████████| 120/120 [00:55<00:00,  2.15it/s]


In [6]:
# 将文本分割成多个块
token_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = token_splitter.split_text(knowledge)

In [7]:
# 自定义向量数据库
class KnowledgeDataBase:
    def __init__(self,chunks) -> None:
        self.model = FlagModel('BAAI/bge-large-zh-v1.5',
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=True)
        self.chunks = [drop_content_from_text(chunk,'�') for chunk in chunks]
        self.db = self.create_db(chunks)

    # 创建数据库
    def create_db(self,chunks):
        print('create db..')
        db = []
        for chunk in tqdm(chunks):
            db.append(self.model.encode(chunk))
        return db

    # 查询最接近的n个文本块
    def search(self,query,n=5):
        query_embedding = self.model.encode(query)
        similarity_scores = [(doc_id, cosine_similarity(query_embedding.reshape(1,-1), doc.reshape(1,-1))[0][0]) 
                            for doc_id, doc in enumerate(self.db)]
        top_n_results = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:n]
        return top_n_results
    

In [8]:
kdb = KnowledgeDataBase(chunks)

create db..


  3%|▎         | 94/3449 [00:52<31:24,  1.78it/s]

In [62]:
# 读取问题并选取最接近的答案和编码
df_test = pd.read_csv(test_path)
df_test['id'] = df_test['question'].apply(lambda x:[id for id,score in kdb.search(x,n=1)][0])
df_test['answer'] = df_test['id'].apply(lambda x:kdb.chunks[x])
df_test['embedding'] = df_test['id'].apply(lambda x:str(list(kdb.db[x]))[1:-2])

In [63]:
df_test['id']

0     1115
1     1623
2     1504
3     1506
4     1571
      ... 
95     817
96     821
97     685
98     694
99     699
Name: id, Length: 100, dtype: int64

In [ ]:
df_test['question'][0],df_test['answer'][0]

In [64]:
# 保存结果
df_test.drop(['id'],axis=1,inplace=True)
df_test.to_csv('result.csv',index=False)